In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

import warnings
warnings.filterwarnings("ignore")

## 1.  Load Data Set

In [3]:
df = sns.load_dataset('titanic')
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [7]:
# Define features and target

features = ['pclass', 'sex', 'age', 'fare']
target = ['survived']

## 2. Check and Fill Missing Value

In [8]:
df[features].isna().sum()

pclass      0
sex         0
age       177
fare        0
dtype: int64

In [9]:
df[target].isna().sum()

survived    0
dtype: int64

In [11]:
# Normality test check for imputing strategy
from scipy.stats import normaltest
normaltest(df['age'].dropna())

NormaltestResult(statistic=18.105032952089758, pvalue=0.00011709599657350757)

In [14]:
dp_statistic, dp_pvalue = normaltest(df['age'].dropna())
if dp_pvalue > 0.05:
    print (f'P-Value: {dp_pvalue}. Asumption Data have normal distribution')
else:
     print (f'P-Value: {dp_pvalue}. Asumption Data NOT have normal distribution')

P-Value: 0.00011709599657350757. Asumption Data NOT have normal distribution


In [15]:
# Since data that we're gonna fill not have 
# normal distribution we're gonna fill with median value

df['age'].median()

28.0

In [16]:
# Fill missing value

df['age'] = df['age'].fillna(df['age'].median())

In [18]:
# Recheck missing value
df[features].isna().sum()

pclass    0
sex       0
age       0
fare      0
dtype: int64

In [23]:
# Define X and y

X = df[features]
y = df[target]


Get Dummies
When one (or more) of the independent variables is a categorical

variable, the most common method of properly including them in the model is

to code them as dummy variables.

In [24]:
# Create Dummy Variabels
X = pd.get_dummies(X, columns=['sex'], drop_first=True)
X

,pclass,age,fare,sex_male
0,3,22.0,7.2500,1
1,1,38.0,71.2833,0
2,3,26.0,7.9250,0
3,1,35.0,53.1000,0
4,3,35.0,8.0500,1
...,...,...,...,...
886,2,27.0,13.0000,1
887,1,19.0,30.0000,0
888,3,28.0,23.4500,0
889,1,26.0,30.0000,1


## 3. Multicollinearity check

In [26]:
# Function to calculate VIF
def calc_vif(X):
    vif = pd.DataFrame()
    vif['variables'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['Acceptable'] = np.where(vif.VIF < 4, 'Yes', 'No') 
    return (vif) 


The reason why VIF acceptable at <4 means that they dont have multicollinearity,

it can be more than 4, but for this studies we take less than 4. The higher the VIF is the most likely they have

multicollinearity.

In [28]:
calc_vif(X)

,variables,VIF,Acceptable
0,pclass,3.728285,Yes
1,age,4.057797,No
2,fare,1.426615,Yes
3,sex_male,2.901604,Yes


Interpretation

Only the age feature has a VIF value slightly above 4. In this case, this value can still be tolerated and the interpretation of the coef can be considered valid later.

In [29]:
# Splitting data
X_train, X_test, y_train, y_test= train_test_split(
    X,
    y,
    stratify = y, # is used to keep the proportion of each class in accordance with the overall data
    test_size = 0.2, # size from test set
    random_state = 0  # is used so that the resulting output remains the same even if it is run repeatedly
)

In [30]:
df['survived'].value_counts()

0    549
1    342
Name: survived, dtype: int64

Target --> survived 

0 --> not survived --> 549

1 -->  survived --> 342

In [32]:
# Check the shapes
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(891, 4)
(712, 4)
(179, 4)


The data got split 80:20 between train & test in the right way

## 5. Logistic regression modelling using statsmodels

Logistic regression is the type of regression analysis used to find the probability of a certain event occurring. It is the best suited type of regression for cases where we have a categorical dependent variable which can take only discrete values. 

In [34]:
sm_logit = sm.Logit(y_train, sm.add_constant(X_train))
result = sm_logit.fit()

Optimization terminated successfully.
         Current function value: 0.444076
         Iterations 6


In [35]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:               survived   No. Observations:                  712
Model:                          Logit   Df Residuals:                      707
Method:                           MLE   Df Model:                            4
Date:                Wed, 13 Oct 2021   Pseudo R-squ.:                  0.3329
Time:                        08:43:47   Log-Likelihood:                -316.18
converged:                       True   LL-Null:                       -473.99
Covariance Type:            nonrobust   LLR p-value:                 4.628e-67
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.7614      0.585      8.142      0.000       3.615       5.908
pclass        -1.1880      0.156     -7.628      0.000      -1.493      -0.883
age           -0.0345      0.008     -4.073      0.0

## 6. Make intrepertation with summary result

Outputs to note:
1. LLR p-value
2. P>|z| (p-value of each variable)
3. coef (Beta or regression coefficient of each variable)

In [37]:
# Interpretation only valid within this range
df.describe().loc[['min', 'max']][['pclass', 'age', 'fare']]

,pclass,age,fare
min,1.0,0.42,0.0000
max,3.0,80.00,512.3292


**The model is the chance of a person surviving the crash of the Titanic**

1. LLR p-value = 4.628e-67 = 0.0000...4628

   - LLR p-value < 0.05, it means we can reject the null hypothesis (H0). It can be said that there is at least one independent variable that has a significant effect on a person's survival rate.
<br><br>

2. P>|z| (Wald test)

   - const = 0.000. p-value < 0.05, reject H0. That is, the model requires an intercept value.
   - pclass = 0.000. p-value < 0.05, reject H0. This means that the passenger ticket class has a significant negative effect on the survival rate (the higher the class number (class 3), the smaller the chance of survival).
   - age = 0.000. p-value < 0.05, reject H0. This means that the age of the passenger has a significant negative effect on the survival rate (the older the passenger, the smaller the chance of survival).
   - fares = 0.547. p-value > 0.05, failed to reject H0. This means that the amount of fare paid by passengers has no significant effect on the survival rate.
   - sex_male = 0.000. p-value < 0.05, reject H0. This means that gender (male passengers) has a significant negative effect on survival rates (Male passengers tend not to survive, because they may give priority to saving female passengers and children).
<br><br>

3. Coef
   - const = 4.7614
   - pclass = -1.1880
   - age = -0.0345
   - fare = 0.0013 (does not need to be interpreted because it has no significant effect on a person's survival rate)
   - sex_male = -2.6591

***Coef interpretation using Odd Ratio (OR)***

OR = exp(beta(c-a))

- If OR > 1, c > a: success rate increases if Xi increases.
- If OR < 1, c > a: success rate decreases as Xi increases.

### pclass


In [38]:
# Interpretation pclass

c = 3
a = 1 # pick c & a within range 1-3, wth c > a
Beta = -1.1880

OR_pclass = np.exp(Beta*(c-a))
print('OR_pclass =', OR_pclass)

OR_pclass_interpretation = 1/OR_pclass
print('OR_pclass_interpretation =', OR_pclass_interpretation)

OR_pclass = 0.0929215212131989
OR_pclass_interpretation = 10.761769576561306



- OR_pclass < 1, success rate increases as Xi (pclass) decreases (3 to 1).
- OR_pclass < 1, 1st class passengers, 10.76 times greater chance of survival than 3rd class passengers.

###  Age

In [40]:
# Interpretaion age

c = 50
a = 40 
Beta = -0.0345

OR_age = np.exp(Beta*(c-a))
print('OR_age =', OR_age)

OR_age_interpretation = 1/OR_age
print('OR_age_interpretation =', OR_age_interpretation)

OR_age = 0.7082203534678
OR_age_interpretation = 1.411989919667659


- OR_age < 1, success rate increases as Xi (age) decreases (age 50 to 40).
- OR_age < 1. passengers aged 40 years have a 1.41 times greater chance of survival than passengers aged 50 years.

### Sex_male

In [43]:
c = 1
a = 0
Beta = -2.6591

OR_sex_male = np.exp(Beta*(c-a))
print('OR_sex_male =', OR_sex_male)

OR_sex_male_interpretation = 1/OR_sex_male
print('OR_sex_male_interpretation =', OR_sex_male_interpretation)

OR_sex_male = 0.07001120348175598
OR_sex_male_interpretation = 14.283428226749269


- OR_sex_male < 1, success rate increases when Xi (sex) is female.
- OR_sex_male < 1, female passengers, 14.2 times more likely to survive than male passengers.

## 7. Check and intrepert model accuracy with test set

In [46]:
# Check the model result
y_predict_proba = result.predict(sm.add_constant(X_test))
y_predict_class = np.where(y_predict_proba > .5, 1, 0)

In [47]:
y_predict_proba

153    0.055234
752    0.069960
610    0.473449
200    0.082047
310    0.945617
         ...   
96     0.184129
440    0.704350
75     0.089984
575    0.109310
143    0.108314
Length: 179, dtype: float64

In [48]:
y_predict_class

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0])

### Evaluation metrics

In [50]:
print('Model accuracy score in the test set:', accuracy_score(y_test, y_predict_class))

Model accuracy score in the test set: 0.7877094972067039


Interpretation of metrics results

Say there are 100 passengers predicted by the model, then 78 passengers will be predicted (survived or not survived) correctly.

### Check the accuracy of model by sklearn

In [53]:
from sklearn.linear_model import LogisticRegression

In [54]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [55]:
accuracy_score(y_test, model.predict(X_test))

0.7877094972067039

For this case, modeling using statsmodels and sklearn using random_state = 0 produces the same accuracy value.